In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.metrics import *

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [8]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 10s 180ms/step - loss: 3.1355 - accuracy: 0.5378 - precision: 0.5136 - recall: 0.2640 - auc: 0.5627
Epoch 2/30
58/58 [==============================] - 10s 177ms/step - loss: 2.0971 - accuracy: 0.6068 - precision: 0.6186 - recall: 0.4206 - auc: 0.6492
Epoch 3/30
58/58 [==============================] - 10s 178ms/step - loss: 1.6785 - accuracy: 0.6386 - precision: 0.6384 - recall: 0.5280 - auc: 0.6926
Epoch 4/30
58/58 [==============================] - 10s 179ms/step - loss: 1.4434 - accuracy: 0.6857 - precision: 0.6516 - recall: 0.7079 - auc: 0.7516
Epoch 5/30
58/58 [==============================] - 10s 178ms/step - loss: 1.2876 - accuracy: 0.7306 - precision: 0.6813 - recall: 0.7991 - auc: 0.7895
Epoch 6/30
58/58 [==============================] - 10s 180ms/step - loss: 1.2185 - accuracy: 0.7459 - precision: 0.7130 - recall: 0.7664 - auc: 0.7737
Epoch 7/30
58/58 [==============================] - 10s 179ms/step - loss: 1.1066 - accu

58/58 [==============================] - 11s 182ms/step - loss: 0.6593 - accuracy: 0.8105 - precision_1: 0.8102 - recall_1: 0.7780 - auc_1: 0.8685
Epoch 24/30
58/58 [==============================] - 11s 182ms/step - loss: 0.6462 - accuracy: 0.8160 - precision_1: 0.8171 - recall_1: 0.7827 - auc_1: 0.8693
Epoch 25/30
58/58 [==============================] - 10s 180ms/step - loss: 0.7126 - accuracy: 0.7481 - precision_1: 0.6972 - recall_1: 0.8178 - auc_1: 0.8157
Epoch 26/30
58/58 [==============================] - 11s 181ms/step - loss: 0.6232 - accuracy: 0.8160 - precision_1: 0.7814 - recall_1: 0.8435 - auc_1: 0.8854
Epoch 27/30
58/58 [==============================] - 10s 178ms/step - loss: 0.6075 - accuracy: 0.8269 - precision_1: 0.8125 - recall_1: 0.8201 - auc_1: 0.8818
Epoch 28/30
58/58 [==============================] - 10s 178ms/step - loss: 0.6451 - accuracy: 0.8028 - precision_1: 0.8351 - recall_1: 0.7220 - auc_1: 0.8667
Epoch 29/30
58/58 [==============================] - 10s 1

58/58 [==============================] - 11s 181ms/step - loss: 0.8266 - accuracy: 0.7634 - precision_3: 0.7477 - recall_3: 0.7477 - auc_3: 0.8324
Epoch 15/30
58/58 [==============================] - 10s 180ms/step - loss: 0.7752 - accuracy: 0.7985 - precision_3: 0.7760 - recall_3: 0.8014 - auc_3: 0.8561
Epoch 16/30
58/58 [==============================] - 10s 181ms/step - loss: 0.9691 - accuracy: 0.6572 - precision_3: 0.6923 - recall_3: 0.4836 - auc_3: 0.7165
Epoch 17/30
58/58 [==============================] - 10s 181ms/step - loss: 1.0552 - accuracy: 0.5301 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.4530
Epoch 18/30
58/58 [==============================] - 11s 184ms/step - loss: 0.9657 - accuracy: 0.5301 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.4861
Epoch 19/30
58/58 [==============================] - 11s 182ms/step - loss: 0.9129 - accuracy: 0.5301 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.4781
Epoch 20/30
58/58 [===============

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 18s 317ms/step - loss: 4.4214 - accuracy: 0.5181 - precision_5: 0.4730 - recall_5: 0.2453 - auc_5: 0.5548
Epoch 2/30
58/58 [==============================] - 18s 318ms/step - loss: 2.6462 - accuracy: 0.5947 - precision_5: 0.5901 - recall_5: 0.4439 - auc_5: 0.6602
Epoch 3/30
58/58 [==============================] - 18s 316ms/step - loss: 2.0129 - accuracy: 0.6933 - precision_5: 0.6486 - recall_5: 0.7547 - auc_5: 0.7457
Epoch 4/30
58/58 [==============================] - 18s 316ms/step - loss: 1.7075 - accuracy: 0.7426 - precision_5: 0.6990 - recall_5: 0.7921 - auc_5: 0.8082
Epoch 5/30
58/58 [==============================] - 19s 319ms/step - loss: 1.4762 - accuracy: 0.8116 - precision_5: 0.7771 - recall_5: 0.8388 - auc_5: 0.8548
Epoch 6/30
58/58 [==============================] - 18s 316ms/step - loss: 1.4306 - accuracy: 0.7251 - precision_5: 0.6606 - recall_5: 0.8505 - auc_5: 0.7765
Epoch 7/30
58/58 [==============================] - 

58/58 [==============================] - 18s 315ms/step - loss: 0.8877 - accuracy: 0.5378 - precision_6: 0.5882 - recall_6: 0.0467 - auc_6: 0.5165
Epoch 23/30
58/58 [==============================] - 18s 316ms/step - loss: 0.8715 - accuracy: 0.5356 - precision_6: 0.6429 - recall_6: 0.0210 - auc_6: 0.5349
Epoch 24/30
58/58 [==============================] - 18s 317ms/step - loss: 0.8584 - accuracy: 0.5345 - precision_6: 1.0000 - recall_6: 0.0070 - auc_6: 0.4976
Epoch 25/30
58/58 [==============================] - 18s 316ms/step - loss: 0.8442 - accuracy: 0.5301 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - auc_6: 0.5342
Epoch 26/30
58/58 [==============================] - 18s 313ms/step - loss: 0.8279 - accuracy: 0.6035 - precision_6: 0.7391 - recall_6: 0.2383 - auc_6: 0.6493
Epoch 27/30
58/58 [==============================] - 18s 313ms/step - loss: 0.8042 - accuracy: 0.7196 - precision_6: 0.7028 - recall_6: 0.6963 - auc_6: 0.7411
Epoch 28/30
58/58 [==============================]

58/58 [==============================] - 18s 315ms/step - loss: 0.8982 - accuracy: 0.7996 - precision_8: 0.7910 - recall_8: 0.7780 - auc_8: 0.8515
Epoch 14/30
58/58 [==============================] - 18s 315ms/step - loss: 0.8248 - accuracy: 0.8248 - precision_8: 0.8221 - recall_8: 0.7991 - auc_8: 0.8829
Epoch 15/30
58/58 [==============================] - 18s 315ms/step - loss: 0.8525 - accuracy: 0.7864 - precision_8: 0.7862 - recall_8: 0.7477 - auc_8: 0.8357
Epoch 16/30
58/58 [==============================] - 18s 314ms/step - loss: 0.8572 - accuracy: 0.7886 - precision_8: 0.8184 - recall_8: 0.7056 - auc_8: 0.7931
Epoch 17/30
58/58 [==============================] - 18s 315ms/step - loss: 0.7959 - accuracy: 0.8050 - precision_8: 0.7894 - recall_8: 0.7967 - auc_8: 0.8152
Epoch 18/30
58/58 [==============================] - 18s 316ms/step - loss: 0.7262 - accuracy: 0.8302 - precision_8: 0.8421 - recall_8: 0.7850 - auc_8: 0.8533
Epoch 19/30
58/58 [==============================] - 18s 3